In [ ]:
import os, json
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util as bsonju

payload = {"role": "cashier", "employee_number": "10001", "name": "John Smith"}

def run(payload):
    role = (payload.get("role") or "").strip()
    if not role:
        return {"ok": False, "reason": "missing_role"}

    load_dotenv()
    uri = os.getenv("MONGODB_URI"); dbn = os.getenv("MONGODB_DATABASE")
    if not uri or not dbn:
        return {"ok": False, "reason": "missing_env"}

    col = MongoClient(uri)[dbn]["sales_invoices"]

    docs = list(col.find({"is_empty": True}, {"employees": 1}))
    if len(docs) != 1:
        return {"ok": False, "reason": "expected_single_empty_form", "found": len(docs)}

    doc = docs[0]; doc_id = doc.get("_id")
    employees = doc.get("employees")
    if isinstance(employees, dict):
        employees = [employees]
    elif not isinstance(employees, list):
        employees = []

    role_key = role.lower()
    idx = next((i for i, e in enumerate(employees)
                if isinstance(e, dict) and (e.get("role") or "").strip().lower() == role_key), None)

    if idx is None:
        return {
            "ok": False,
            "reason": "role_not_in_empty_form",
            "role": role,
            "available_roles": [str((e or {}).get("role", "")).strip() for e in employees if isinstance(e, dict)]
        }

    current = employees[idx] if isinstance(employees[idx], dict) else {}
    employees[idx] = {
        "role": current.get("role") or role,
        "employee_number": payload.get("employee_number", current.get("employee_number")),
        "name": payload.get("name", current.get("name")),
    }

    res = col.update_one({"_id": doc_id}, {"$set": {"employees": employees}})
    return {
        "ok": True,
        "matched_count": res.matched_count,
        "modified_count": res.modified_count,
        "updated_employee": employees[idx],
    }

out = run(payload)
print(bsonju.dumps(out, indent=2, ensure_ascii=False))


{
  "ok": false,
  "reason": "role_not_in_empty_form",
  "role": "cashiers",
  "available_roles": [
    "cashier",
    "recorder"
  ]
}
